In [1]:
import gym

from gym import Env
from gym.spaces import Discrete,Box,Dict,Tuple,MultiBinary,MultiDiscrete

from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

import os
import pygame
import random as rn
import numpy as np

In [2]:
class TribounceEnv(Env):
    def __init__(self):

        self.width,self.height = 1000,700
        self.moveRight= False 
        self.moveLeft = False 

        self.jump = False
        self.jumpe = False
        self.goingUp = True
        self.goingUpSpeed = 11.0
        self.gravitationalAccelartion = 0.2
        
        self.playerSpeed = 6

        self.clock = pygame.time.Clock()

        self.playerPos = [self.width/2,self.height-200]
        self.playerHeight = 60
        
        self.enemyStats = []
        
        self.enemySpeed = 5

        self.kills = 0

        self.fps = 120
        self.fpsCount = 0
        self.time = 0  

        self.reward = 0   

        self.action_space = Discrete(3)# have 3 valuse to be controlled
        self.observation_space = Box(low=0,high=self.width,shape=(2,2))
        self.state = np.array([self.playerPos,[0,self.width]])



    def state(self,action):

        nearEnemy = []

        if action == 0:
            self.moveLeft = True
            self.moveRight = False
        elif action == 1:
            self.moveLeft = False
            self.moveRight = True
        if action == 2 and self.height-205<self.playerPos[1]<self.height-195:
            self.jump = True
        elif action != 2 and self.height-202<self.playerPos[1]<self.height-198:
            self.jump = False

        if self.jump and round(self.goingUpSpeed,1) > -11.0 + self.gravitationalAccelartion:
            self.goingUpSpeed -=self.gravitationalAccelartion
        elif self.jump :
            self.jump = False
            self.jumpe = False
            self.goingUpSpeed = 11.0
            self.playerPos[1] = self.height-200

        if self.jump:
            self.playerPos[1]-=self.goingUpSpeed

        if self.jumpe and round(self.goingUpSpeed,1) > -11.6 + self.gravitationalAccelartion:
            self.goingUpSpeed -=self.gravitationalAccelartion
            self.jump = False
        elif self.jumpe :
            self.jumpe = False
            
            self.goingUpSpeed = 11.0
            self.playerPos[1] = self.height-200

        if self.playerPos[0]> self.width - self.playerHeight/2:
            self.playerPos[0] = self.width-self.playerHeight/2
        elif self.playerPos[0]< self.playerHeight/2:
            self.playerPos[0] = self.playerHeight/2

        if self.time > 2 and self.fpsCount%(1.7*self.fps)== 0:
            ran = rn.randint(0,1)
            if ran == 0:
                self.enemyStats.append([[self.width-40,self.height-200],1,ran])
            else:
                self.enemyStats.append([[40,self.height-200],1,ran])

        
        for e in self.enemyStats:
            # print(self.playerPos,[e[0][0],e[0][1]-40], self.goingUpSpeed<0)

            if (e[0][0]-5 - 40<=self.playerPos[0]<=e[0][0]+5+40) and (e[0][1]  - 1 - 80<= self.playerPos[1] <= e[0][1] + 1 - 80) and self.goingUpSpeed<0 :
                e[1] = 0
                self.jumpe = True 
                self.goingUpSpeed = 10.0
                self.kills +=1
                self.reward +=10
        

            # if (e[0]- triwe/2 <= trix <= e[0]+triwe/2) and (self.height - rectw - triwe/2 <= triy <= self.height - rectw + triwe/2):
            elif (e[0][0]-2-40-self.playerHeight/2 <= self.playerPos[0] <= e[0][0]+2+40+self.playerHeight/2) and (self.height-200 - 80 - 2 <= self.playerPos[1]<=self.height-200 + 2) :
                done= True
            
            else:
                done = False
                
                
                
            
            if e[0][0] <= 40:
                e[2] = 1 
            elif e[0][0] >= self.width - 40:
                e[2] = 0 

            if e[2] == 1:
                e[0][0] += self.enemySpeed
            elif e[2] == 0:
                e[0][0] -= self.enemySpeed

            if e[1] == 0:
                self.enemyStats.remove(e)


            nearEnemy.append(e[0]-self.playerPos[0])

        self.fpsCount +=1
        if self.fpsCount%self.fps == 0:
            self.enemySpeed+=0.1
            self.time+=1
            self.reward += self.time

        nearEnemy.sort()
        
        self.state = np.array([self.playerPos,[len(self.enemyStats),nearEnemy]])
        info = {}
        self.clock.tick(self.fps) 
       
        _ = True

        return self.state , self.reward , done,_ , info
    
    def render(self):
        pass


        
                    
    def reset(self):
        self.state = np.array([self.playerPos,[0,self.width]])

        self.moveRight = False 
        self.moveLeft = False 

        self.jump = False
        self.jumpe = False
        self.goingUp = True
        self.goingUpSpeed = 11.0
        
        self.playerSpeed = 6

        self.playerPos = [self.width/2,self.height-200]
        
        self.enemyStats = [[[0,0],0,0]]
        self.enemySpeed = 5

        self.kills = 0
        self.fpsCount = 0
        self.time = 0  

        self.reward = 0  
        return self.state
    
   

In [4]:
env = TribounceEnv()


In [10]:
env.observation_space.sample()

array([[592.86523, 860.2477 ],
       [393.09454, 125.39939]], dtype=float32)

In [11]:
env = DummyVecEnv([lambda:env])


c:\Users\arya2\AppData\Local\Programs\Python\Python311\Lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


In [12]:
log_path = os.path.join('Training','Logs')
model = PPO('MlpPolicy',env,verbose=1)

Using cuda device


In [13]:
model.learn(total_timesteps=1000)

NotImplementedError: 